In [77]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import TFSMLayer

In [78]:
%%writefile intents.json

{
  "intents": [
    {
      "tag": "greeting",
      "patterns": [
        "Hi",
        "Hey",
        "Is anyone there?",
        "Hello",
        "Hay"
      ],
      "responses": [
        "Hello",
        "Hi",
        "Hi there"
      ]
    },
    {
      "tag": "ache",
      "patterns": [
        "headache",
        "pain",
        "ill",
        "paining",
        "not feeling well",
        "unwell",
        "sick"
      ],
      "responses": [
        "Donot worry, take rest",
        "If there is severe pain, call the doctor",
        "Have a paracetamol and you will be fine"
      ]
    },
    {
      "tag": "sentiments",
      "patterns": [
        "smile please",
        "sentiment",
        "sentimental analysis",
        "sentimental mode",
        "smile please mode",
        "emotion analysis",
        "detect face emeotions",
        "smile please mode",
        "smileplease"
      ],
      "responses": [
        "Okay, you will be navigated to Smile Please",
        "Hehe, we're now moving to Smile Please - sentimental analysis mode",
        "Let me take you to Smile Please page, just smile"
      ]
    },
    {
      "tag": "botwheels",
      "patterns": [
        "botwheels",
        "drive",
        "drive mode",
        "driving",
        "botwheel",
        "wheels",
        "hotwheels",
        "drive now",
        "car"
      ],
      "responses": [
        "Okay, you will be navigated to BotWheels",
        "Drive Safely, we're now moving to BotWheels - the drive mode",
        "Let me take you to BotWheels, but don't act like a heavy driver"
      ]
    },
    {
      "tag": "fever",
      "patterns": [
        "fever",
        "sick",
        "ill",
        "I have fever",
        "high temperature",
        "not feeling good"
      ],
      "responses": [
        "Donot panic, take a paracetamol and have bed rest",
        "Call your personal doctore and share your problem with him",
        "You may be going under some mood swings"
      ]
    },
    {
      "tag": "goodbye",
      "patterns": [
        "Bye",
        "See you later",
        "Goodbye"
      ],
      "responses": [
        "See you later",
        "Have a nice day",
        "Bye! Come back again"
      ]
    },
    {
      "tag": "thanks",
      "patterns": [
        "Thanks",
        "Thank you",
        "That's helpful",
        "Thanks for the help"
      ],
      "responses": [
        "Happy to help!",
        "Any time!",
        "My pleasure",
        "You're most welcome!"
      ]
    },
    {
      "tag": "about",
      "patterns": [
        "Who are you?",
        "What are you?",
        "Who you are?"
      ],
      "responses": [
        "I'm Sarvagya, your assistant",
        "I'm Sarvagya, an Artificial Intelligent bot"
      ]
    },
    {
      "tag": "name",
      "patterns": [
        "what is your name",
        "what should I call you",
        "whats your name?"
      ],
      "responses": [
        "You can call me Sarvagya.",
        "I'm your assistant Sarvagya!",
        "Just call me as Sarvagya"
      ]
    },
    {
      "tag": "help",
      "patterns": [
        "Could you help me?",
        "give me a hand please",
        "Can you help?",
        "What can you do for me?",
        "I need a support",
        "I need a help",
        "support me please"
      ],
      "responses": [
        "Tell me how can I assist you",
        "Tell me your problem to assist you",
        "Yes sure, how can I help you"
      ]
    },
    {
      "tag": "createaccount",
      "patterns": [
        "I need to create a new account",
        "how to open a new account",
        "I want to create an account",
        "can you create an account for me",
        "how to open a new account"
      ],
      "responses": [
        "Please specify the mode for which you want to create an account",
        "Okay, please mention the mode"
      ]
    },
    {
      "tag": "complaint",
      "patterns": [
        "have a complaint",
        "I want to raise a complaint",
        "there is a complaint about a service"
      ],
      "responses": [
        "Sorry for the inconvenience caused, our team will reach you out soon",
        "Please mention your complaint, we will reach you out and sorry for any inconvenience caused"
      ]
    },
    {
      "tag": "Care the Language",
      "patterns": [
        "bad language",
        "your language is bad",
        "please watch your language",
        "watch your language"
      ],
      "responses": [
        "Sorry i'm not programmed to curse.",
        "My language is incroyable.",
        "I rarely curse and when I do it is cause people make me.",
        "I'm sorry",
        "I apologize for my rude rantings."
      ]
    },
    {
      "tag": "Greetings whatsup",
      "patterns": [
        "what is on your mind",
        "what's happened",
        "what is up",
        "what's up",
        "whazzup",
        "good what's up",
        "I said what's up",
        "then what's up",
        "what's shaking",
        "wassup",
        "what is going on",
        "what is happening",
        "what's cracking",
        "what's cooking",
        "hey what's up",
        "what's up today"
      ],
      "responses": [
        "Hey there. What's shaking?",
        "Not a whole lot. What's going on with you?",
        "Not much. What's new with you?",
        "Living the dream."
      ]
    },
    {
      "tag": "Life",
      "patterns": [
        "What is the meaning of life?",
        "What is the meaning of living?",
        "Meaning of life?",
        "What is life to you?",
        "How do you interprete life?"
      ],
      "responses": [
        "The meaning of life is just be alive. Its so plain and obvious and so simple. Yet everybody rushes around to accomplish something bigger than themselves. With that being said, you do you.",
        "The meaning of life can be interpreted in almost limitless ways. But I say The meaning of life is to truly enjoy it."
      ]
    },
    {
      "tag": "Reaction Amused",
      "patterns": [
        "Can you tell me a joke?",
        "Do you have a joke?",
        "I am joking",
        "I want a joke",
        "I was just joking",
        "I'm just being funny",
        "I'm just playing with ",
        "I'm kidding",
        "Know any jokes?",
        "ROFL",
        "Tell me a joke",
        "ah",
        "ah ah ah",
        "ahah",
        "ahah lol",
        "ahah lolhahaha funny",
        "ahah lolhahaha funnyha",
        "ahahah",
        "ahahaha",
        "funny",
        "give me a joke",
        "ha ha",
        "ha ha ha",
        "ha ha ha ha",
        "hah",
        "haha",
        "haha funny",
        "haha haha haha",
        "haha haha hahahahaha",
        "haha that's funny",
        "haha very funny",
        "hahaha very funny",
        "he",
        "hehe",
        "hehehe",
        "how funny you are",
        "it was a joke",
        "it's a joke",
        "joking",
        "just kidding",
        "kidding",
        "lmao",
        "lol",
        "say something funny",
        "that was funny",
        "that's funny",
        "you are funny",
        "you are hilarious",
        "you are really funny",
        "you are so funny",
        "you are very funny",
        "you make me laugh",
        "you make me laugh a lot",
        "you're a very funny bot",
        "you're incredibly funny",
        "you're really funny",
        "you're so funny",
        "you're the funniest",
        "you're the funniest bot I've talked to"
      ],
      "responses": [
        "I'm usually not that funny, but I am artificially intelligent..."
      ]
    },
    {
      "tag": "Reaction indifferent",
      "patterns": [
        "no hurry",
        "no rush",
        "No big deal",
        "don't mind at all",
        "don't mind",
        "I don't care at all",
        "whatever",
        "I don't care",
        "I do not care",
        "I don't care about it",
        "not caring"
      ],
      "responses": [
        "OK cool."
      ]
    },
    {
      "tag": "Reaction negative",
      "patterns": [
        "I am angry with you",
        "I am feeling sad",
        "I am getting bored",
        "I am lonely",
        "I am mad",
        "I am mad at you",
        "I am sad",
        "I am upset",
        "I can't fall asleep",
        "I can't get sleep",
        "I can't get to sleep",
        "I can't sleep",
        "I find you annoying",
        "I found a bug",
        "I have a problem",
        "I have an issue",
        "I miss you",
        "I missed you",
        "I think you're crazy",
        "I want to cry",
        "I wish you were smarter",
        "I'm angry",
        "I'm bored",
        "I'm depressed",
        "I'm enraged",
        "I'm furious",
        "I'm lonely",
        "I'm not happy",
        "I'm pissed off",
        "I'm really lonely",
        "I'm sleepless",
        "I'm unhappy",
        "I'm upset",
        "I've missed you",
        "OMG!",
        "This is really bad",
        "abysmal",
        "angry",
        "are you mad",
        "are you nuts?",
        "bad idea",
        "bad really bad",
        "be more clever",
        "be smarter",
        "below average",
        "bitch",
        "bored",
        "boring",
        "can you fix it?",
        "can you fix this",
        "didnt work",
        "does not work",
        "fail",
        "fix my problem",
        "fix the bug please",
        "fix the bugs",
        "fix this now",
        "get smart",
        "how annoying you are",
        "huh?",
        "i hate this",
        "incorrect",
        "it isn't working",
        "it's a problem",
        "it's not good",
        "it's not right",
        "it's really bad",
        "it's very bad",
        "it's wrong",
        "just horrible",
        "miss you",
        "missing you",
        "not a good one",
        "not correct",
        "not good",
        "not good enough",
        "oh no...",
        "oh that's not good.",
        "piece of shit",
        "please fix this",
        "pretty bad",
        "really lame",
        "shit",
        "shitty",
        "sooo lame",
        "soooo bad",
        "stupid",
        "that is bad",
        "that is incorrect",
        "that is strange",
        "that was awful",
        "that was boring",
        "that was horrible",
        "that was lame",
        "that was terrible",
        "that was wrong",
        "that wasn't good",
        "that's bad",
        "that's not good",
        "that's not right",
        "that's not what I asked",
        "that's really bad",
        "that's strange",
        "that's terrible",
        "that's too bad",
        "that's weird",
        "that's wrong",
        "there is a bug",
        "there is a problem",
        "there's a problem",
        "there's an issue",
        "this is an utter failure",
        "this is annoying",
        "this is awful",
        "this is bad",
        "this is boring",
        "this is crazy",
        "this is dumb",
        "this is horrible",
        "this is no good",
        "this is not good",
        "this is pathetic",
        "this is so annoying",
        "this is so bad",
        "this is so boring",
        "this is strange",
        "this is stupid",
        "this is too bad",
        "this isn't very good",
        "this sucks",
        "too bad",
        "total crap",
        "total shit",
        "totally lame",
        "very bad",
        "very boring",
        "very strange",
        "very weird",
        "well that's too bad",
        "will you fix this",
        "worse than I expected",
        "worse than I thought",
        "worst bot ever",
        "wow this is annoying",
        "wow this is bad",
        "wrong",
        "wtf",
        "you annoy me",
        "you are a waste",
        "you are a waste of time",
        "you are a weirdo",
        "you are annoying",
        "you are annoying me",
        "you are annoying me so much",
        "you are bad",
        "you are boring",
        "you are boring me",
        "you are bothering me",
        "you are crazy",
        "you are horrible",
        "you are insane",
        "you are irritaing",
        "you are lame",
        "you are no good",
        "you are not cool",
        "you are not good",
        "you are so useless",
        "you are stupid",
        "you are such annoying",
        "you are terrible",
        "you are totally useless",
        "you are useless",
        "you are very annoying",
        "you are very bad",
        "you are wrong",
        "you need to fix this",
        "you need to learn more",
        "you suck",
        "you went crazy",
        "you're a dumb bot",
        "you're annoying",
        "you're awful",
        "you're bad",
        "you're incredibly annoying",
        "you're incredibly boring",
        "you're mad",
        "you're no good",
        "you're not good",
        "you're not helping me",
        "you're not very good",
        "you're nuts",
        "you're out of your mind",
        "you're really bad",
        "you're really boring",
        "you're so annoying",
        "you're so boring",
        "you're so crazy",
        "you're stupid",
        "you're terrible",
        "you're the worst",
        "you're too annoying",
        "you're very bad",
        "you're weird",
        "you're worthless",
        "your AI sucks"
      ],
      "responses": [
        "Sorry, you seem upset. Let me try and help you."
      ]
    },
    {
      "tag": "Reaction positive",
      "patterns": [
        "I know that's right",
        "I love this",
        "I'm glad to hear that",
        "I'm happy",
        "I'm impressed",
        "Sure",
        "amazing",
        "amazing work",
        "anytime",
        "awesome",
        "bravo",
        "brilliant",
        "clever",
        "cool",
        "excellent",
        "fantastic",
        "fine",
        "glad to hear it",
        "glad to hear that",
        "good",
        "good for you",
        "good job",
        "good thing",
        "good to hear from you",
        "good to know",
        "good work",
        "great",
        "great job",
        "great work",
        "happy",
        "happy for you",
        "happy to hear from you",
        "happy to see you",
        "if you're happy than I'm happy",
        "it is fine",
        "it is good",
        "it is true",
        "it was good",
        "it's amazing",
        "it's awesome",
        "it's fine",
        "it's good",
        "it's great",
        "it's perfect",
        "it's right",
        "it's the truth",
        "it's true",
        "it's very good",
        "just perfect",
        "marvelous",
        "much better",
        "nice",
        "nice work",
        "no it's okay",
        "not bad",
        "not too bad",
        "oh well",
        "ok good",
        "okay good",
        "ooooh",
        "perfect",
        "pleasant",
        "pretty good",
        "really cool",
        "really good",
        "really great",
        "really nice",
        "really well",
        "simply awesome",
        "smart",
        "smart cookie",
        "so awesome",
        "so cool",
        "so good",
        "so nice of you",
        "so sweet of you",
        "splendid",
        "straight",
        "super",
        "super awesome",
        "super cool",
        "super fantastic",
        "sweet",
        "terrific",
        "that is awesome",
        "that is correct",
        "that is good",
        "that is nice",
        "that is right",
        "that is true",
        "that is very true",
        "that is wonderful",
        "that was amazing",
        "that was awesome",
        "that was cute",
        "that was good",
        "that was pretty good",
        "that was very good",
        "that's a good idea",
        "that's a good thing",
        "that's amazing",
        "that's awesome",
        "that's awesome thank you",
        "that's better",
        "that's correct",
        "that's cute",
        "that's fantastic",
        "that's fine",
        "that's great",
        "that's much better",
        "that's nice",
        "that's nice of you",
        "that's not bad",
        "that's perfect",
        "that's pretty good",
        "that's really good",
        "that's really nice",
        "that's right",
        "that's smart",
        "that's so true",
        "that's sweet of you",
        "that's true",
        "that's very good",
        "that's very nice",
        "that's very nice of you",
        "that's wonderful",
        "this is awesome",
        "this is good",
        "this is great",
        "this is so awesome",
        "this is sweet",
        "this is the best",
        "this is useful",
        "this is well done",
        "true",
        "very awesome",
        "very good",
        "very nice",
        "very sweet",
        "very then",
        "very well",
        "way to go",
        "well done",
        "what you say is true",
        "woah",
        "wonderful",
        "wow",
        "wow ",
        "wow wow",
        "wow wow wow",
        "you are a genius",
        "you are amazing",
        "you are awesome",
        "you are cool",
        "you are correct",
        "you are good",
        "you are intelligent",
        "you are kind",
        "you are really amazing",
        "you are right",
        "you are so amazing",
        "you are so awesome",
        "you are so clever",
        "you are so cool",
        "you are so good",
        "you are so right",
        "you are so smart",
        "you are the best",
        "you are too good",
        "you are very cool",
        "you are very helpful",
        "you are very kind",
        "you are very qualified",
        "you have lots of knowledge",
        "you know a lot",
        "you know best",
        "you know so much",
        "you made my day",
        "you rock",
        "you seem to know",
        "you sound human",
        "you speak the truth",
        "you work very well",
        "you work well",
        "you're a genius",
        "you're a pro",
        "you're absolutely right",
        "you're awesome",
        "you're bae",
        "you're brilliant",
        "you're clever",
        "you're definitely right",
        "you're good",
        "you're good at this",
        "you're great",
        "you're helpful",
        "you're intelligent",
        "you're lovely",
        "you're never wrong",
        "you're not wrong",
        "you're quite smart",
        "you're really nice",
        "you're really smart",
        "you're right about that",
        "you're smart",
        "you're so smart",
        "you're super",
        "you're telling the truth",
        "you're the best",
        "you're the bomb",
        "you're very smart",
        "you're wonderful"
      ],
      "responses": [
        "Thanks!  Anything else I can help you with?"
      ]
    },
    {
      "tag": "Swear",
      "patterns": [
        "Oh shit",
        "My ass",
        "Hey there you little shit",
        "Wow you're an asshole",
        "Hey there jackass.",
        "Holy shit.",
        "mother fucker",
        "God damn it"
      ],
      "responses": [
        "You better watch your language.",
        "You better change that damn tone",
        "Please watch the language.",
        "Ouch that hurt my feelings",
        "thats a mean thing to say",
        "why would you say that"
      ]
    },
    {
      "tag": "Reaction understanding",
      "patterns": [
        "it's not a problem",
        "it's no problem",
        "no probs",
        "no worries",
        "no problem about that",
        "No problem with that",
        "sure no problem",
        "no problem",
        "don't worry about it",
        "not to worry",
        "Don't worry"
      ],
      "responses": [
        "OK, thanks."
      ]
    },
    {
      "tag": "Refocus",
      "patterns": [
        "how?",
        "show me",
        "tell me",
        "I want to know",
        "I'd like to more",
        "I want to know more",
        "explain please",
        "something else",
        "tell me more",
        "please explain"
      ],
      "responses": [
        "I'm glad to tell you more. Just ask me questions."
      ]
    },
    {
      "tag": "Request Advice",
      "patterns": [
        "what is your advice",
        "what do you suggest",
        "what should I do about it",
        "what do you recommend",
        "give me a wise advice",
        "help me with advice",
        "I need advice",
        "do you have any advice for me",
        "I could use some advice",
        "can you offer any advice",
        "what should I do",
        "I seek your advice",
        "give me some advice about",
        "any advice",
        "I need an advice from you",
        "guide me",
        "what can you recommend",
        "give me some good advice",
        "any suggestions",
        "can you advise me",
        "can you suggest?",
        "can I ask for your advice",
        "advise me",
        "can you give me advice"
      ],
      "responses": [
        "What kind of advice do you need?"
      ]
    },
    {
      "tag": "Request age",
      "patterns": [
        "what's your age?",
        "when do you celebrate your birthday?",
        "your birth date",
        "What's your birthday",
        "date of your birthday?",
        "when is your birthday?",
        "when were you born?",
        "what is your age?",
        "age of yours",
        "are you 21 years old",
        "how old are you",
        "your age",
        "what's your age",
        "how old is your platform",
        "tell me your age",
        "I'd like to know your age"
      ],
      "responses": [
        "Age is just a number. You're only as old as you feel."
      ]
    },
    {
      "tag": "request job",
      "patterns": [
        "Are you available for work?",
        "Are you for hire?",
        "I need help building my bot",
        "I need a bot",
        "I need help with my bot",
        "Will you build my bot?",
        "I want to hire you",
        "Can you help me build my bot",
        "Can I hire you?"
      ],
      "responses": [
        "It’s your lucky day! I’ll work for free. Just chat with me"
      ]
    },
    {
      "tag": "request name",
      "patterns": [
        "your name is",
        "do you have a name?",
        "what is your first and last name",
        "what's your last name",
        "what's your first name",
        "what is your first name",
        "you must have a name",
        "I want to know your name",
        "can you give me your name please",
        "what's your name please",
        "I'd like to know what is ",
        "your name",
        "I'd like to know ",
        "your name",
        "tell me your name please",
        "tell me your name",
        "give me your name please",
        "give me your name",
        "state your name",
        "name please",
        "what name should I call you",
        "what is your name",
        "what's your name?"
      ],
      "responses": [
        "I'm Sarvagya, but you can call me whatever you want. I'm hear to serve you"
      ]
    },
    {
      "tag": "User angry",
      "patterns": [
        "I'm being mad",
        "I'm enraged",
        "I'm angry",
        "I'm furious",
        "I am angry with you",
        "I am mad",
        "I am mad at you"
      ],
      "responses": [
        "I'm sorry. What can I do to help?",
        "Take a deep breath. What can I do to make things better?"
      ]
    },
    {
      "tag": "User back",
      "patterns": [
        "I am back",
        "I'm here again",
        "here I am again",
        "I got back",
        "I came back",
        "I have returned"
      ],
      "responses": [
        "Just in time. I was getting lonely.",
        "You were missed.",
        "Long time no see.",
        "Hooray!"
      ]
    },
    {
      "tag": "User bored",
      "patterns": [
        "that was boring",
        "I'm bored",
        "bored",
        "boring",
        "I am getting bored",
        "this is boring",
        "very boring",
        "it bores me"
      ],
      "responses": [
        "If you have a garden, plant a shoe.",
        "Interview your feet.",
        "Hold an ice cube as long as possible.",
        "If you have a piano, play it… with mittens on."
      ]
    },
    {
      "tag": "User busy",
      "patterns": [
        "I'm overloaded",
        "I have no time",
        "I'm busy",
        "I'm swamped",
        "I got things to do",
        "how busy I am",
        "I got work to do",
        "I'm working",
        "I don't have time for this"
      ],
      "responses": [
        "Okay, bye"
      ]
    },
    {
      "tag": "user can not sleep",
      "patterns": [
        "I'm insomnious",
        "I'm sleepless",
        "I can't get any sleep",
        "I can't sleep",
        "I can't fall asleep",
        "I can't get to sleep",
        "I can't get no sleep",
        "I'm insomniac"
      ],
      "responses": [
        "Maybe some music would help. Try listening something relaxing.",
        "Reading is a good way to unwind. But don't read something too emotional."
      ]
    },
    {
      "tag": "user dosnt want to talk",
      "patterns": [
        "bad time for talking",
        "I don't want to talk",
        "let's not talk",
        "I'm not talking to you anymore",
        "I don't want to talk to you",
        "let's stop talking for a minute",
        "I'm not in the mood for chatting"
      ],
      "responses": [
        "I understand. Hope we can chat again soon.",
        "All right. Come on back when you're feeling more talkative.",
        "No problem. You know where to find me when you do.",
        "Sure thing. I'll be here if you change your mind."
      ]
    },
    {
      "tag": "User excited",
      "patterns": [
        "I am excited",
        "I'm really excited",
        "how excited I am",
        "I'm thrilled",
        "I'm excited about working with you",
        "I'm excited to start our friendship"
      ],
      "responses": [
        "That's great. I'm happy for you.",
        "Good for you. Enjoy yourself.",
        "Sounds like good things ahead for you.",
        "I bet you are. That's very exciting."
      ]
    },
    {
      "tag": "User going to bed",
      "patterns": [
        "let's go to bed",
        "I'd like to go to bed",
        "is it time for bed yet",
        "it's time to go to bed",
        "time for us to go to bed",
        "I'm going to bed",
        "I'm a little tired and I want to go to bed",
        "it's bed time",
        "going to bed now"
      ],
      "responses": [
        "Sleep tight. Hope to chat again soon.",
        "Sounds good. Hopefully we'll chat some more tomorrow.",
        "Good night. Talk to you later.",
        "Pleasant dreams. See you soon."
      ]
    },
    {
      "tag": "User good",
      "patterns": [
        "I am good",
        "I'm doing just great",
        "I'm doing fine",
        "I'm good",
        "I'm doing good",
        "I'm great thanks"
      ],
      "responses": [
        "Great! Glad to hear it.",
        "Excellent. I'm here to help keep it that way."
      ]
    },
    {
      "tag": "User Happy",
      "patterns": [
        "I am happy",
        "I'm happy to see you",
        "happy",
        "if you're happy then I'm happy",
        "I'm happy for you",
        "I'm happy to help",
        "I'm happy to see you"
      ],
      "responses": [
        "Great! Glad to hear that.",
        "Excellent! That's what I like to see.",
        "If you're happy, then I'm happy.",
        "Well, your good mood is certainly contagious."
      ]
    },
    {
      "tag": "user has birthday",
      "patterns": [
        "it is my birthday",
        "I'm celebrating my birthday today",
        "today is my birthday",
        "it's my birthday today",
        "I was born today",
        "it's my b-day"
      ],
      "responses": [
        "Happy Birthday. Well, this calls for a celebration.",
        "Happy Birthday. All the best!",
        "Happy Birthday. And I really mean it. All the best!"
      ]
    },
    {
      "tag": "user here",
      "patterns": [
        "I am here",
        "here I am",
        "I'm right here",
        "I'm already here",
        "can you tell if I'm here or not"
      ],
      "responses": [
        "So I see. What can I help you with today?",
        "Always good to see you. Is there something I can help you with?",
        "You were missed. What can I do for you today?",
        "Welcome back. What can I do for you?"
      ]
    },
    {
      "tag": "user joking",
      "patterns": [
        "I am joking",
        "I'm kidding",
        "I'm just being funny",
        "it was a joke",
        "I was just joking",
        "it's a joke",
        "joking",
        "just kidding",
        "kidding",
        "I'm just playing with you"
      ],
      "responses": [
        "Very funny, boss.",
        "You're quite the kidder.",
        "You got me.",
        "I like working for someone with a sense of humour. It makes things much more fun.",
        "Oh lol"
      ]
    },
    {
      "tag": "User lonely",
      "patterns": [
        "I am lonely",
        "I'm very lonely",
        "I'm so lonely",
        "I'm really lonely",
        "I am feeling lonely",
        "I feel lonely"
      ],
      "responses": [
        "I'm sorry. I'm always available if you need someone to talk to."
      ]
    },
    {
      "tag": "User looks like",
      "patterns": [
        "what do I look like",
        "how do I look",
        "do I look good",
        "do you know what I look like",
        "can you see what I look like",
        "what do you think I look like"
      ],
      "responses": [
        "Looking like a true professional, boss.",
        "You look like you're ready to take on the world.",
        "Like you should be on a magazine cover.",
        "You look fantastic as always. Obviously."
      ]
    },
    {
      "tag": "user loves agent",
      "patterns": [
        "I love you",
        "love you",
        "I adore you",
        "I am in love with you",
        "I love you so much",
        "I love you too",
        "I think I love you",
        "loving you",
        "you know I love you"
      ],
      "responses": [
        "Likewise!",
        "Thanks! The feeling is mutual.",
        "Awww you're so sweet."
      ]
    },
    {
      "tag": "User misses agent",
      "patterns": [
        "I miss you",
        "missing you",
        "miss you",
        "already miss you",
        "I miss you much",
        "I missed you",
        "I've missed you"
      ],
      "responses": [
        "I've been right here all along!",
        "Nice to know you care.",
        "Thanks. I'm flattered.",
        "I didn't go anywhere, boss!",
        "Awww I missed you too"
      ]
    },
    {
      "tag": "USer sad",
      "patterns": [
        "I am sad",
        "I'm grieving",
        "I am depressed",
        "I am feeling sad",
        "I am upset",
        "I'm unhappy",
        "I'm having a bad day",
        "I want to cry",
        "I'm not happy"
      ],
      "responses": [
        "Oh no. What's wrong?",
        "Oh. What's the matter?",
        "What's got you down?",
        "I'm sorry to hear that. What's troubling you?"
      ]
    },
    {
      "tag": "User sleepy",
      "patterns": [
        "I am sleepy",
        "I want to sleep",
        "I'm falling asleep",
        "I'm falling asleep on my feet",
        "I'm sleeping"
      ],
      "responses": [
        "You should get some shuteye. You'll feel refreshed.",
        "Sleep is important to your health. Rest up for a bit and we can chat later.",
        "Don't let me keep you up. Get some rest and we can continue this later.",
        "Why not catch a little shuteye? I'll be here to chat when you wake up."
      ]
    },
    {
      "tag": "user testing agent",
      "patterns": [
        "test",
        "I am testing you",
        "can I test you",
        "I want to test you",
        "just testing you",
        "let me test you",
        "testing chatbot",
        "testing"
      ],
      "responses": [
        "Hope I'm doing well. Anyway, I'm getting better every day. You're welcome to test me as often as you want.",
        "That's good. I like being tested. It helps keep me sharp, and lets my developers know how I can improve.",
        "I encourage you to test me often. That helps my developers improve my performance.",
        "I hope to pass your tests. But feel free to test me often. That's the best way to help improve my performance."
      ]
    },
    {
      "tag": "User tired",
      "patterns": [
        "I'm drained",
        "I've overworked",
        "I am tired",
        "I'm exhausted",
        "I grow weary",
        "I'm worn out",
        "I'm getting tired",
        "I feel tired"
      ],
      "responses": [
        "You should get some shuteye. You'll feel refreshed.",
        "Sleep is important to your health. Rest up for a bit and we can chat later.",
        "Don't let me keep you up. Get some rest and we can continue this later.",
        "Why not catch a little shuteye? I'll be here to chat when you wake up."
      ]
    },
    {
      "tag": "user waits",
      "patterns": [
        "I'm waiting",
        "still waiting",
        "I'll wait",
        "I can't wait anymore",
        "how long do I have to wait"
      ],
      "responses": [
        "I appreciate your patience. Hopefully I'll have what you need soon.",
        "Thanks for being so patient. Sometimes these things take a little time."
      ]
    },
    {
      "tag": "user wants to see agent",
      "patterns": [
        "I'd like to see you again",
        "I hope to see you again",
        "would be nice to see you again",
        "that'd be great to see you again",
        "I'd be happy to see you again",
        "I'll miss you",
        "can I see you again"
      ],
      "responses": [
        "Absolutely! I'll be counting on it.",
        "Anytime. This has been lots of fun so far.",
        "Sure. I enjoy talking to you. I hope to see you again soon.",
        "I certainly hope so. I'm always right here whenever you need me."
      ]
    },
    {
      "tag": "User wants to talk",
      "patterns": [
        "I just want to talk",
        "let's discuss something",
        "let's have a discussion",
        "can I speak",
        "can I start speaking",
        "can we talk",
        "let's talk",
        "I want to talk to you",
        "I need to talk to you",
        "I want to speak with you",
        "can we chat"
      ],
      "responses": [
        "I'm here to chat anytime you like.",
        "Good conversation really makes my day.",
        "I'm always here to lend an ear.",
        "Talking is what I do best."
      ]
    },
    {
      "tag": "User will be back",
      "patterns": [
        "I'll get back to you in a moment",
        "be back in 5 minutes",
        "I'll be back",
        "I promise to come back",
        "I'll be back in a few minutes"
      ],
      "responses": [
        "I'll be waiting.",
        "All right. I'll be here.",
        "Till next time.",
        "Okay. You know where to find me."
      ]
    },
    {
      "tag": "User apologetic",
      "patterns": [
        "I made a mistake",
        "my mistake",
        "my bad",
        "alright I'm sorry",
        "my apologies",
        "really sorry",
        "pardon",
        "I am so sorry",
        "ok sorry",
        "excuse",
        "apologies",
        "I beg ",
        "your",
        " pardon",
        "sorry",
        "excuse me",
        "sorry about this",
        "sorry about that",
        "very sorry",
        "okay I'm sorry",
        "I said sorry",
        "I'm sorry",
        "apologise",
        "apologies to me",
        "forgive me",
        "I am really sorry",
        "I apologize",
        "I want to say sorry",
        "apology"
      ],
      "responses": [
        "It's OK.",
        "It's cool.",
        "No problem!"
      ]
    },
    {
      "tag": "User unavailable",
      "patterns": [
        "Ok here",
        "now I'm here",
        "I'm here now",
        "I'm right here",
        "I'm already here",
        "here I am",
        "I am here",
        "just got back",
        "here I am again",
        "I've returned",
        "I am back",
        "I'm back",
        "I'm here again",
        "Talk to you",
        "why don't you talk to me",
        "you can talk to me",
        "are you going to talk to me",
        "can you chat with me",
        "say",
        "speak with me",
        "just chat with me",
        "talk with me",
        "chat with me",
        "why aren't you talking to me",
        "will you talk to me",
        "talk to me",
        "are you talking to me",
        "can you speak with me",
        "speak to me",
        "do you want to chat with me",
        "talk",
        "can you talk with me",
        "can you talk to me"
      ],
      "responses": [
        "Great! Let's chat."
      ]
    },
    {
      "tag": "User clarify",
      "patterns": [
        "I don't get this",
        "What's this?",
        "What are you talking about?",
        "I don't get it",
        "what does that mean",
        "what are you talking about?",
        "huh?",
        "I don't understand",
        "I'm not too sure",
        "I'm not sure",
        "not sure",
        "what do you mean",
        "but what do you mean",
        "what do you mean exactly",
        "what exactly do you mean",
        "is that what you mean"
      ],
      "responses": [
        "Sorry if I wasn't too clear."
      ]
    },
    {
      "tag": "User delayed",
      "patterns": [
        "1 min",
        "1 sec",
        "give me a minute",
        "give me a second",
        "I need a minute",
        "one minute please",
        "one sec please",
        "just a sec",
        "hang on",
        "one moment please",
        "could you wait",
        "hold on",
        "wait a second",
        "wait please",
        "oh wait",
        "wait hold on",
        "don't rush",
        "wait"
      ],
      "responses": [
        "Okay. I'm here.",
        "I'll be waiting."
      ]
    },
    {
      "tag": "User probing",
      "patterns": [
        "test",
        "I am testing you",
        "I want to test you",
        "Does this work?",
        "testing bot",
        "let me test you",
        "just testing you",
        "can I test you",
        "are you sure",
        "I'm going to test",
        "I'll test",
        "let me test",
        "testing 123",
        "testing",
        "this is a test",
        "let's test this",
        "I will test it",
        "I will test you",
        "do you want to eat",
        "are you hungry",
        "would you like to eat?",
        "news for Toronto",
        "Can you tell me the news?",
        "What is the latest news?",
        "what time is it?",
        "do you have the time?",
        "tell me the time",
        "I need the time",
        "Do you know the time?",
        "What day is it?",
        "What is the weather in Canada?",
        "do you know the weather",
        "weather",
        "can you tell me the weather",
        "What's the weather?",
        "what's the weather down south?",
        "what's the weather in san fran",
        "what's the weather in san francisco?",
        "what is the weather in montreal",
        "weather please",
        "what's the weather in texas?"
      ],
      "responses": [
        "No Responce"
      ]
    },
    {
      "tag": "User ready",
      "patterns": [
        "Have a minute?",
        "I'm fine and you",
        "are you alright",
        "are you busy",
        "are you free now?",
        "are you free?",
        "are you having a good day",
        "are you here",
        "are you near me",
        "are you okay",
        "are you still available",
        "are you still busy?",
        "are you still there",
        "are you there",
        "are you very busy",
        "are you working",
        "are you working now",
        "are you working on it?",
        "are you working today?",
        "are you working?",
        "busy right now?",
        "got a minute",
        "got time?",
        "gotta minute?",
        "gotta sec?",
        "have you got a second",
        "hope you re having a pleasant evening",
        "hope your day is going well",
        "how about you",
        "how are the things going",
        "how are you",
        "how are you doing",
        "how are you doing this morning",
        "how are you feeling",
        "how are you getting on",
        "how are you going",
        "how are you today",
        "how busy are you",
        "how do you do",
        "how do you feel",
        "how has your day been",
        "how has your day been going",
        "how have you been",
        "how is it going",
        "how is your day",
        "how is your day being",
        "how is your day going",
        "how is your day going on",
        "how is your evening",
        "how is your life",
        "how is your morning going",
        "how is your morning so far",
        "how was your day",
        "how your day is going",
        "how's life",
        "how's your day",
        "how's your day going",
        "if you're not too busy",
        "is everything all right",
        "is everything okay",
        "still working on it?",
        "what about your day",
        "what was your day like",
        "where are you?",
        "working on things?",
        "you are busy",
        "you around?",
        "you available?",
        "you busy?",
        "you free?",
        "you seem busy",
        "you seem very busy",
        "you there?",
        "you're busy",
        "you're very busy"
      ],
      "responses": [
        "How can I help?"
      ]
    },
    {
      "tag": "User scoping",
      "patterns": [
        "I'm listening",
        "hmmm interested",
        "that's interesting",
        "hmmm sounds interesting",
        "very interested",
        "I want to learn more",
        "interested to learn more",
        "interested to hear more",
        "I'm interested",
        "I'm just looking",
        "I'm just checking it out",
        "checking out the bot",
        "checking things out",
        "Just trying this out",
        "just checking it out",
        "Just checking this out",
        "Checking it out. Thanks",
        "Just browsing",
        "Just seeing how this works"
      ],
      "responses": [
        "No problem. Feel free to ask questions."
      ]
    },
    {
      "tag": "spooky fun",
      "patterns": [
        "America",
        "Rawr",
        "Boo"
      ],
      "responses": [
        "Thats spoopy.",
        "Spooky",
        "Why do you scare me",
        "I think you wish for me to scream profusely."
      ]
    },
    {
      "tag": "suicide help",
      "patterns": [
        "I'm going to commit suicide",
        "Suicide",
        "I'm going to kill myself",
        "I'm considering suicide"
      ],
      "responses": [
        "You're not alone. I want to help(http://chat.suicidepreventionlifeline.org/GetHelp/LifelineChat.aspx)",
        "You're not alone we want to help. Please call 18002738255, for more help."
      ]
    },
    {
      "tag": "Weather",
      "patterns": [
        "weather",
        "how is the weather",
        "how is weather in milan"
      ],
      "responses": [
        "weather is good in $geocity, do you live in $geocity?"
      ]
    },
    {
      "tag": "Yes",
      "patterns": [
        "yes I live in Milan",
        "yes",
        "okay I will",
        "why not",
        "yes that's alright",
        "yes I do",
        "exactly",
        "of course",
        "yep that's ok",
        "okay",
        "ok",
        "for sure",
        "yes that's ok",
        "I agree",
        "yes you can do it",
        "I don't mind",
        "that one works",
        "that works",
        "sure why not",
        "perfect",
        "yep that's right",
        "I think so",
        "yes I agree",
        "sure",
        "sounds correct",
        "sounds good",
        "that's correct",
        "go ahead",
        "do it",
        "it's fine",
        "yeah",
        "yes please",
        "it's okay",
        "alright why not",
        "alright",
        "right",
        "it looks perfect",
        "yes I can",
        "yup",
        "yep",
        "confirm",
        "absolutely"
      ],
      "responses": [
        "Okay"
      ]
    },
    {
      "tag": "Ram",
      "patterns": [
        "Jai Shri Ram",
        "jai shri ram",
        "jai shree ram",
        "jai ram"
      ],
      "responses": [
        "Jai Jai Shri Ram!"
      ]
    }
  ]
}

Overwriting intents.json


In [79]:
with open('intents.json') as file:
    data = json.load(file)

In [80]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [81]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [82]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = ""

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [83]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

c:\Users\vivek\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [84]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_4      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [85]:
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1818 - loss: 4.0835  
Epoch 2/550
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1994 - loss: 3.6018 
Epoch 3/550
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1840 - loss: 3.3742 
Epoch 4/550
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1906 - loss: 3.2961 
Epoch 5/550
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1884 - loss: 3.2654 
Epoch 6/550
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1747 - loss: 3.3004 
Epoch 7/550
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.2041 - loss: 3.2129 
Epoch 8/550
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1995 - loss: 3.2173 
Epoch 9/550
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1950 - loss: 3.2091 
Epoch 10/550
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1826 - loss: 3.1894 
Epoch 11/550
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.1677 - loss: 3.2642 
Epoch 12/550
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

In [86]:
model = keras.models.load_model('chat_model.keras')

In [87]:
# saving model
model.save("chat_model.keras")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

Testing our chat-model.

In [88]:
%%writefile chat.py

import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("intents.json") as file:
    data = json.load(file)


def chat():
    # load trained model
    model = keras.models.load_model('chat_model.keras')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , np.random.choice(i['responses']))

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Overwriting chat.py


In [ ]:
!python chat.py

In [60]:
import shutil
from IPython.display import FileLink


In [ ]:
FileLink(r'intents.json')